In [1]:
from ebel.manager.neo4j.n4j_meta import Neo4jClient
from ebel.manager.neo4j.bel import Neo4jBel
import os

## Validate bel files and get report or complete json file

In [3]:
import os
import glob
from ebel.validate import validate_bel_file

# Define the folder containing the BEL files
data_folder = "bel-files"
report_folder = "validation-reports"

# Create a folder for reports if it doesn't exist
os.makedirs(report_folder, exist_ok=True)

# Find all .bel files in the folder
bel_files = glob.glob(os.path.join(data_folder, "*.bel"))

# Loop through and validate each file
for bel_file in bel_files:
    # Extract the base filename without extension
    base_name = os.path.splitext(os.path.basename(bel_file))[0]
    
    # Define report filename
    report_file = os.path.join(report_folder, f"{base_name}_report.xlsx")
    
    # Validate the BEL file
    validate_bel_file(
        bel_file,
        reports=report_file,
        json_file=True,
        force_json=True
    )


Check BEL for MESHPP: 100%|█████████████████████████████████████████| 81/81 [00:04<00:00, 18.83it/s]


In [4]:
paths = []
for path, _, files in os.walk("bel-files"):
    for file in files:
        print(file)
        if file.endswith(".bel.json"):
            print(path)
            paths.append(os.path.join(path, file))

.DS_Store
curated_bel_files.bel
curated_bel_files.bel.json
bel-files
BEL to EXCEL-checkpoint.ipynb


In [5]:
neo = Neo4jClient(
    uri="bolt://localhost:7687", database="neo4j", user="neo4j", password="PKG_v4@1234"
)
# neo.delete_everything()


In [6]:
#Load all graphs
n4jbel = Neo4jBel(client=neo)
for path in paths:
    n4jbel.import_json(input_path=path, update_from_protein2gene=False) #Maria added True

print("Done")
# Add source property to all relationships
neo.execute("MATCH ()-[r]->() SET r.source = 'PKG'") ##Change it to "manual-curation" for new triples!!

print("Done")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: bel)} {position: line: 1, column: 45, offset: 44} for query: 'MATCH (n) RETURN elementId(n) AS node_id, n.bel as bel'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: 

Done
Done


In [ ]:
neo.